In [1]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import statsmodels.api as sm

In [2]:
from TDSCoinbaseData import TDSCoinbaseData
from TDSTickGenerator import TDSTickGenerator
from TDSTransactionTracker import TDSTransactionTracker
import logging
logging.getLogger().setLevel(level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

In [3]:
# instantiate our data interface
cb_obj = TDSCoinbaseData(cache_path='../data', notebook_logging=True)
# Set the start and end date
start_date = '20200701'
end_date = '20200930'

# List all of the products to be used in the strategy. Be sure to list all products you may use, these can't be updated later
products = ['BTC-USD', 'ETH-USD', 'BTC-EUR', 'ETH-EUR', 'LTC-USD', 'LTC-EUR']

In [4]:
MAX_TRAINNING_SIZE = 360
SIZE_LIMIT_REACHED = False

In [5]:
def append_ticks(tick, usd_btc, eth_btc, ltc_btc, 
                 max_r=SIZE_LIMIT_REACHED, scale_btc=1000000, scale_eth=10000, scale_ltc=10000):
    x = tick.p.btc_usd
    y = tick.p.eth_usd
    z = tick.p.ltc_usd
    usd_btc = usd_btc.append({'open': scale_btc/x.open, 'close': scale_btc/x.close, 
                      'volume': x.volume, 'datetime': tick.datetime}, ignore_index=True)
    eth_btc = eth_btc.append({'open': scale_eth * y.open/x.open, 'close': scale_eth *  y.close/x.close, 
                      'volume': y.volume, 'datetime': tick.datetime}, ignore_index=True)
    ltc_btc = ltc_btc.append({'open': scale_ltc * z.open/x.open, 'close': scale_ltc * z.close/x.close, 
                      'volume': z.volume, 'datetime': tick.datetime}, ignore_index=True)
    # Checks if size limit has been reached
    if (SIZE_LIMIT_REACHED or len(usd_btc.index) > MAX_TRAINNING_SIZE):
        usd_btc.iloc[1:]
        eth_btc.iloc[1:]
        ltc_btc.iloc[1:]
        max_r = True
        
    return(usd_btc, eth_btc, ltc_btc, max_r)

In [6]:
def calculate_time(pddatetime, numtime, plus=True):
    if plus:
        return pddatetime.to_pydatetime() + timedelta(minutes=numtime)
    else: 
        return pddatetime.to_pydatetime() - timedelta(minutes=numtime)

# frame -- dataframe
# minute -- the number of minutes to be predicted
def predict(frame, minute=3) :
    frame = frame.set_index('datetime')
    y = frame['open'].resample('1T').mean()
    l = frame['close'].resample('1T').mean()
    v = frame['volume'].resample('1T').mean()
    merged = pd.merge(y, l, on='datetime', how="outer")
    frame = pd.merge(merged, v, on='datetime', how="outer")

    mod = sm.tsa.VARMAX(frame[['open', 'close', 'volume']], order=(1, 0), error_cov_type='diagonal')
    results = mod.fit(maxiter=50, disp=False)
    #print(results.summary().tables[1])
    newtime = calculate_time(tick.datetime, minute)
    pred = results.get_prediction(start=pd.to_datetime(frame.index[-1]), end=newtime, dynamic=False)
    pred_ci = pred.conf_int()
    #print(pred.predicted_mean)
#     print(frame.iloc[-1])
    return (pred)

In [7]:
# Instantiate our transaction tracker. As required, we start with 1.0 BTC
trans_tracker = TDSTransactionTracker(start_date, end_date, holdings={'BTC' : 1.0})

# Instantiate a tick generator
tick_gen = TDSTickGenerator(cb_obj, products, start_date, end_date, interval=60)
tick = tick_gen.get_tick()

# Dataframe for history
d = {'open': [], 'close': [], 'volume': [], 'datetime': []}
usd_btc = pd.DataFrame(data=d)
eth_btc = pd.DataFrame(data=d)
ltc_btc = pd.DataFrame(data=d)

# loads initial value for testing
for i in range(0, 150):
    tick = tick_gen.get_tick()
    (usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED) = append_ticks(tick, usd_btc, eth_btc, ltc_btc)

In [8]:
def get_coin_ratio(tick, holdings):
    btc = holdings.get("BTC") if "BTC" in holdings else 0
    eth = tick.p.eth_usd.close/tick.p.btc_usd.close * holdings.get("ETH") if "ETH" in holdings else 0
    ltc = tick.p.ltc_usd.close/tick.p.btc_usd.close * holdings.get("LTC") if "LTC" in holdings else 0
    usd = holdings.get("USD")/tick.p.btc_usd.close if "USD" in holdings else 0
    res = np.array([usd, eth, ltc, btc])
    return sum(res), res/sum(res)

In [9]:
def trade_ratio(curr_port, ideal_port, tick, trans_tracker, curr_hold_in_BTC):
    gap = (ideal_port - curr_port)[1:]
    gap_in_usd = gap * curr_hold_in_BTC * tick.p.btc_usd.close
    names = np.array(['ETH-USD', 'LTC-USD', 'BTC-USD'])
    names = names[np.argsort(gap_in_usd)]
    gap_in_usd = np.sort(gap_in_usd)
    # holdings = trans_tracker.get_holdings()
    
    for idx in range(len(names)):
        product = names[idx]
        change = gap_in_usd[idx]
        attr = getattr(tick.p, product.lower().replace('-', '_'))
        # curr_hold = trans_tracker.get_holdings().get(product) if product in 
        if change < 0:
            valid_vol = min(- change / attr.close, attr.volume * 0.5)
#             print(product[:3], trans_tracker.get_holdings().get(product[:3]))
            valid_vol = min(valid_vol, trans_tracker.get_holdings().get(product[:3])) // 0.000001 / 1000000
#             print(- change / attr.close, attr.volume * 0.5)
            trans_tracker.make_trade(tick, product, 'sell', valid_vol)
        elif change > 0:
            valid_amt = min(change, trans_tracker.get_holdings().get("USD"))
#             print(attr.volume)
            valid_amt = min(valid_amt, attr.volume * attr.close * 0.5) // 0.000001 / 1000000
#             print(valid_amt)
            trans_tracker.make_trade(tick, product, 'buy', valid_amt)


In [10]:
def trade_eth(tick, bu, eu, be, ee, has_usd):
        # Amount of ETH and BTC to trade at most due to volume restriction
        num_eth = min(tick.p.eth_usd.volume, tick.p.eth_eur.volume) / 2
        num_btc = num_eth * ee / be
        if (num_btc > min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2):
            num_btc = min(num_eth * ee / be, min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2)
            num_eth = num_btc * be / ee

        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, has_usd))
        # print(trans_tracker.get_holdings())
        # Sell Bitcoin for EUR
        trans_tracker.make_trade(tick, 'BTC-EUR', 'sell', min(num_btc, trans_tracker.get_holdings().get("BTC")))
        # print(trans_tracker.get_holdings())
        # Buy ETH with EUR
        trans_tracker.make_trade(tick, 'ETH-EUR', 'buy', min(num_eth * ee, trans_tracker.get_holdings().get("EUR")))
        # print(trans_tracker.get_holdings())
        # Sell ETH with USD
        trans_tracker.make_trade(tick, 'ETH-USD', 'sell', trans_tracker.get_holdings().get("ETH"))
        # print(trans_tracker.get_holdings())
        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, trans_tracker.get_holdings().get("USD")))
        # print(trans_tracker.get_holdings())

In [11]:
def trade_ltc(tick, bu, lu, be, le, has_usd):
        # Amount of LTC and BTC to trade at most due to volume restriction
        num_ltc = min(tick.p.ltc_usd.volume, tick.p.ltc_eur.volume) / 2
        num_btc = num_ltc * le / be
        if (num_btc > min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2):
            num_btc = min(num_ltc * le / be, min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2)
            num_ltc = num_btc * be / le

        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, has_usd))
        # print(trans_tracker.get_holdings())
        # Sell Bitcoin for EUR
        trans_tracker.make_trade(tick, 'BTC-EUR', 'sell', min(num_btc, trans_tracker.get_holdings().get("BTC")))
        # print(trans_tracker.get_holdings())
        # Buy LTC with EUR
        trans_tracker.make_trade(tick, 'LTC-EUR', 'buy', min(num_ltc * le, trans_tracker.get_holdings().get("EUR")))
        # print(trans_tracker.get_holdings())
        # Sell LTC with USD
        trans_tracker.make_trade(tick, 'LTC-USD', 'sell', trans_tracker.get_holdings().get("LTC"))
        # print(trans_tracker.get_holdings())
        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, trans_tracker.get_holdings().get("USD")))
        # print(trans_tracker.get_holdings())

In [19]:
def trades(tick, trans_tracker, usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED, scale_btc=1000000, scale_eth=10000, scale_ltc=10000):
    
    # Get current prices 
    x_price = scale_btc / tick.p.btc_usd.close
    y_price = scale_eth * tick.p.eth_usd.close / tick.p.btc_usd.close
    z_price = scale_ltc * tick.p.ltc_usd.close / tick.p.btc_usd.close

    curr_hold_in_BTC, curr_port = get_coin_ratio(tick, trans_tracker.get_holdings())
    # print(count, curr_hold_in_BTC)
    
    # Get predicted values
    pred_usd = predict(usd_btc, 1)
    pred_eth = predict(eth_btc, 1)
    pred_ltc = predict(ltc_btc, 1)
    
    # Find which one increases the most
    changes = np.array([(pred_usd.predicted_mean["close"][-1]-x_price)/x_price, 
               (pred_eth.predicted_mean["close"][-1]-y_price)/y_price, 
               (pred_ltc.predicted_mean["close"][-1]-z_price)/z_price, 
                       0])
    print(curr_port, changes)
    # Obtain ideal portfolio
    if max(changes) - sum(curr_port * changes) > 0.005:
        ideal_port = np.zeros_like(changes)
        ideal_port[changes.argmax(0)] = 1

        # print(count, curr_port, ideal_port)
        trade_ratio(curr_port, ideal_port, tick, trans_tracker, curr_hold_in_BTC)
    
    (usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED) = append_ticks(tick, usd_btc, eth_btc, ltc_btc)

In [24]:
hr=0
# count = 0
benchmark = 1.4
while tick is not None:
    curr_hold_in_BTC, curr_port = get_coin_ratio(tick, trans_tracker.get_holdings())
    # count += 1
    # print(count, curr_hold_in_BTC)
    if (tick and not hr == 0 and hr != tick.datetime.to_pydatetime().hour):
        print("Time:", tick.datetime.to_pydatetime().hour)
        print("holdings", trans_tracker.get_holdings())
        hr = tick.datetime.to_pydatetime().hour
        # 10% chance of attempting to trade
#         no trade for this tick
    if (not tick.p.btc_usd.volume and not tick.p.eth_usd.volume and not tick.p.ltc_usd.volume):
        continue

    bu = tick.p.btc_usd.close
    eu = tick.p.eth_usd.close
    lu = tick.p.ltc_usd.close
    be = tick.p.btc_eur.close
    ee = tick.p.eth_eur.close
    le = tick.p.ltc_eur.close

    has_usd = trans_tracker.get_holdings().get("USD")

    # Check exchange ratios for BTC - ETH and BTC - LTC
    be_ratio = (be * eu) / (ee * bu) * 100 - 100
    bl_ratio = (be * lu) / (le * bu) * 100 - 100

    # Check if it is an arbitrage opportunity for BTC - ETH or BTC - LTC
    if (be_ratio < benchmark and bl_ratio < benchmark):
        trades(tick, trans_tracker, usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED)
        tick = tick_gen.get_tick()
        continue

    # Check which would guarantee larger profit
    if (be_ratio > bl_ratio):
        trade_eth(tick, bu, eu, be, ee, has_usd)
    else:
        trade_ltc(tick, bu, lu, be, le, has_usd)

    tick = tick_gen.get_tick()

[0. 0. 0. 1.] [-0.00202034 -0.01176276 -0.0103526   0.        ]
[0. 0. 0. 1.] [-0.00202034 -0.01176276 -0.0103526   0.        ]
[0. 0. 0. 1.] [-0.00202143 -0.01176384 -0.01035369  0.        ]
[0. 0. 0. 1.] [-0.00210259 -0.01188774 -0.00995635  0.        ]
[0. 0. 0. 1.] [-0.00266848 -0.01188225 -0.01051779  0.        ]
[0. 0. 0. 1.] [-0.003094   -0.01191189 -0.01046215  0.        ]
[0. 0. 0. 1.] [-0.00254785 -0.01184983 -0.01015912  0.        ]
[0. 0. 0. 1.] [-0.00254675 -0.01193583 -0.01039701  0.        ]
[0. 0. 0. 1.] [-0.00258952 -0.0119782  -0.01115565  0.        ]
[0. 0. 0. 1.] [-0.00254675 -0.01167452 -0.01111325  0.        ]
[0. 0. 0. 1.] [-0.00202363 -0.01167894 -0.01059462  0.        ]
[0. 0. 0. 1.] [-0.00202253 -0.01215657 -0.01035478  0.        ]
[0. 0. 0. 1.] [-0.00172533 -0.01225383 -0.01172943  0.        ]
[0. 0. 0. 1.] [-0.00133161 -0.01242936 -0.01205367  0.        ]
[0. 0. 0. 1.] [-0.0013689  -0.01259655 -0.01232832  0.        ]
[0. 0. 0. 1.] [-0.00111557 -0.01208534 -

[0. 0. 0. 1.] [-0.00461731 -0.01172231 -0.01030079  0.        ]
[0. 0. 0. 1.] [-0.00430914 -0.01141633 -0.01023386  0.        ]
[0. 0. 0. 1.] [-0.00374434 -0.01159771 -0.01039057  0.        ]
[0. 0. 0. 1.] [-0.00365332 -0.0115074  -0.01030016  0.        ]
[0. 0. 0. 1.] [-0.00377834 -0.01136964 -0.01042434  0.        ]
[0. 0. 0. 1.] [-0.00389349 -0.01174569 -0.00982069  0.        ]
[0. 0. 0. 1.] [-0.00383756 -0.01164657 -0.01024393  0.        ]
[0. 0. 0. 1.] [-0.00383427 -0.01159969 -0.01024066  0.        ]
[0. 0. 0. 1.] [-0.00407335 -0.00987028 -0.00999948  0.        ]
[0. 0. 0. 1.] [-0.00388033 -0.01103427 -0.00980761  0.        ]
[0. 0. 0. 1.] [-0.00415231 -0.01134796 -0.00959882  0.        ]
[0. 0. 0. 1.] [-0.00436507 -0.01147187 -0.00909097  0.        ]
[0. 0. 0. 1.] [-0.00443636 -0.01149898 -0.00916192  0.        ]
[0. 0. 0. 1.] [-0.00504721 -0.01171264 -0.00904989  0.        ]
[0. 0. 0. 1.] [-0.0050845  -0.01192432 -0.00908702  0.        ]
[0. 0. 0. 1.] [-0.00504941 -0.01162747 -

[0. 0. 0. 1.] [-0.00144896 -0.01341342 -0.01002489  0.        ]
[0. 0. 0. 1.] [-0.00144896 -0.0137602  -0.01002489  0.        ]
[0. 0. 0. 1.] [-0.00144896 -0.01389018 -0.01097931  0.        ]
[0. 0. 0. 1.] [-0.00144896 -0.0143232  -0.01145583  0.        ]
[0. 0. 0. 1.] [-0.00127349 -0.01427984 -0.01128212  0.        ]
[0. 0. 0. 1.] [-0.00108705 -0.01396595 -0.01204956  0.        ]
[0. 0. 0. 1.] [-0.00108705 -0.01470148 -0.01204956  0.        ]
[0. 0. 0. 1.] [-0.00041916 -0.01460479 -0.01162688  0.        ]
[0. 0. 0. 1.] [-0.00022176 -0.01436696 -0.0116695   0.        ]
[0. 0. 0. 1.] [ 0.00043516 -0.01380584 -0.01054406  0.        ]
[0. 0. 0. 1.] [-0.00019654 -0.01399616 -0.01140675  0.        ]
[0. 0. 0. 1.] [-0.00010003 -0.01429018 -0.01131132  0.        ]
[0. 0. 0. 1.] [ 6.44776677e-05 -1.44305568e-02 -1.11486616e-02  0.00000000e+00]
[0. 0. 0. 1.] [ 6.99611387e-05 -1.36900687e-02 -1.09052476e-02  0.00000000e+00]
[0. 0. 0. 1.] [ 0.00018511 -0.01452769 -0.01079136  0.        ]
[0. 0. 0

[0. 0. 0. 1.] [-0.00207517 -0.01120726 -0.00921153  0.        ]
[0. 0. 0. 1.] [-0.00244476 -0.01135557 -0.00909966  0.        ]
[0. 0. 0. 1.] [-0.00291085 -0.0108581  -0.00860408  0.        ]
[0. 0. 0. 1.] [-0.00297556 -0.01087864 -0.00866842  0.        ]
[0. 0. 0. 1.] [-0.00293498 -0.01127474 -0.00910757  0.        ]
[0. 0. 0. 1.] [-0.00257636 -0.01205223 -0.00970913  0.        ]
[0. 0. 0. 1.] [-0.00261255 -0.01152187 -0.0090268   0.        ]
[0. 0. 0. 1.] [-0.00270248 -0.01134947 -0.00911615  0.        ]
[0. 0. 0. 1.] [-0.00253359 -0.01135644 -0.00966667  0.        ]
[0. 0. 0. 1.] [-0.00253469 -0.01100867 -0.00966776  0.        ]
[0. 0. 0. 1.] [-0.00253359 -0.01109482 -0.00966667  0.        ]
[0. 0. 0. 1.] [-0.0026981  -0.01143231 -0.00983     0.        ]
[0. 0. 0. 1.] [-0.00260268 -0.01142492 -0.00973527  0.        ]
[0. 0. 0. 1.] [-0.00268055 -0.0107606  -0.00933388  0.        ]
[0. 0. 0. 1.] [-0.00260159 -0.0105513  -0.00925544  0.        ]
[0. 0. 0. 1.] [-0.00260159 -0.01033293 -

[0. 0. 0. 1.] [-0.00290208 -0.01111124 -0.01051065  0.        ]
[0. 0. 0. 1.] [-0.00290208 -0.01163451 -0.01051065  0.        ]
[0. 0. 0. 1.] [-0.00271674 -0.0113636  -0.01032673  0.        ]
[0. 0. 0. 1.] [-0.00288453 -0.01144275 -0.0100151   0.        ]
[0. 0. 0. 1.] [-0.00305781 -0.01113481 -0.00946919  0.        ]
[0. 0. 0. 1.] [-0.00325083 -0.01115173 -0.00966097  0.        ]
[0. 0. 0. 1.] [-0.00328812 -0.01110144 -0.00969802  0.        ]
[0. 0. 0. 1.] [-0.00295253 -0.01094309 -0.00984338  0.        ]
[0. 0. 0. 1.] [-0.00295362 -0.01111873 -0.00984447  0.        ]
[0. 0. 0. 1.] [-0.00277815 -0.01103197 -0.00967022  0.        ]
[0. 0. 0. 1.] [-0.00245682 -0.01088784 -0.00959045  0.        ]
[0. 0. 0. 1.] [-0.00257307 -0.01130838 -0.00970587  0.        ]
[0. 0. 0. 1.] [-0.00277596 -0.01137871 -0.00966804  0.        ]
[0. 0. 0. 1.] [-0.00277486 -0.0112032  -0.00990622  0.        ]
[0. 0. 0. 1.] [-0.00277815 -0.01133729 -0.00990948  0.        ]
[0. 0. 0. 1.] [-0.00237896 -0.01085427 -

...

[1.93184005e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00437494 -0.01352946 -0.01411499  0.        ]
[1.93205287e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00448461 -0.01363812 -0.01422358  0.        ]
[1.93205287e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00448461 -0.01363812 -0.01422358  0.        ]
[1.93210821e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00451312 -0.01366637 -0.01425182  0.        ]
[1.9329323e-11 0.0000000e+00 0.0000000e+00 1.0000000e+00] [-0.00493754 -0.01343463 -0.01395928  0.        ]
[1.9329323e-11 0.0000000e+00 0.0000000e+00 1.0000000e+00] [-0.00493754 -0.01339112 -0.014197    0.        ]
[1.93293443e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00493864 -0.0137837  -0.01277008  0.        ]
[1.93341815e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00518759 -0.01368254 -0.01301707  0.        ]
[1.93363344e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00529836 -0.01396632 -0.01312696  0.        ]


[1.93660989e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00682715 -0.01478666 -0.01725362  0.        ]
[1.93589377e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00645976 -0.01459615 -0.01689009  0.        ]
[1.93624428e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00663961 -0.01486147 -0.01706806  0.        ]
[1.93564592e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00633254 -0.01429602 -0.01629047  0.        ]
[1.93604763e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00653872 -0.01450054 -0.01720487  0.        ]
[1.93660133e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00682276 -0.01482579 -0.01701259  0.        ]
[1.93703554e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00704539 -0.01500316 -0.01699618  0.        ]
[1.93703554e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00704539 -0.01500316 -0.01699618  0.        ]
[1.93703554e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00704539 -0.01500316 -0.01699618  0.  

[1.93314962e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00504941 -0.01376303 -0.01596843  0.        ]
[1.93325404e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00510315 -0.0138163  -0.01602158  0.        ]
[1.93330306e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00512837 -0.0138413  -0.01509837  0.        ]
[1.93331371e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00513385 -0.01402065 -0.0151038   0.        ]
[1.93379975e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.0053839  -0.01418153 -0.01535134  0.        ]
[1.93386585e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.0054179  -0.01421523 -0.01538499  0.        ]
[1.93384666e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00540803 -0.01442273 -0.01537522  0.        ]
[1.93379975e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.0053839  -0.01439882 -0.01535134  0.        ]
[1.93454206e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00576555 -0.01455981 -0.01572916  0.  

[1.94159892e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00937916 -0.01055044 -0.0159878   0.        ]
[1.94140764e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00928155 -0.01080457 -0.01612867  0.        ]
[1.9407889e-11 0.0000000e+00 0.0000000e+00 1.0000000e+00] [-0.0089657  -0.01062104 -0.015815    0.        ]
[1.94201602e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00959191 -0.01076294 -0.01643688  0.        ]
[1.94144202e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.0092991  -0.01047047 -0.01471746  0.        ]
[1.94080608e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00897448 -0.01036609 -0.01558582  0.        ]
[1.9402049e-11 0.0000000e+00 0.0000000e+00 1.0000000e+00] [-0.0086674  -0.01032324 -0.01528079  0.        ]
[1.94028861e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00871017 -0.01067349 -0.01532328  0.        ]
[1.93976284e-11 0.00000000e+00 0.00000000e+00 1.00000000e+00] [-0.00844148 -0.01062496 -0.01505638  0.        ]


KeyboardInterrupt: 

In [26]:
print(trans_tracker.get_sharpe_ratio())

RISK FREE DAILY RETURN : 
0.00016438356164383562
ACTUAL DAILY RETURN : 
0.00015141235671801436
EXCESS STD : 
0.002119091538562206
-0.0061211159073487524


In [27]:
trans_tracker.get_holdings()

{'BTC': 1.0138204709656093, 'USD': 1.7780439520720392e-07}

In [25]:
trans_tracker.get_trades()

[{'date': '20200701',
  'timestamp': 1593591840,
  'side': 'sell',
  'size': 1.0,
  'price': 9165.5,
  'product': 'BTC-USD',
  'holdings': {'BTC': 0.0, 'USD': 9149.0021}},
 {'date': '20200702',
  'timestamp': 1593702660,
  'side': 'buy',
  'size': 1.0085045608117418,
  'price': 9071.85,
  'product': 'BTC-USD',
  'holdings': {'BTC': 1.0066892526022806, 'USD': 0.0}},
 {'date': '20200704',
  'timestamp': 1593884100,
  'side': 'sell',
  'size': 1.006689,
  'price': 9166.97,
  'product': 'BTC-USD',
  'holdings': {'BTC': 2.5260228064816204e-07, 'USD': 9211.676944177805}},
 {'date': '20200705',
  'timestamp': 1593928800,
  'side': 'buy',
  'size': 1.0156483854571516,
  'price': 9069.75,
  'product': 'BTC-USD',
  'holdings': {'BTC': 1.0138204709656093, 'USD': 1.7780439520720392e-07}}]

In [ ]:
trans_tracker.dump_trades('run_2.json')

In [17]:
curr_hold_in_BTC, curr_port = get_coin_ratio(tick, trans_tracker.get_holdings())
curr_hold_in_BTC

0.989925687965194